# scraping github 

search github to find assistive technology.

In [1]:
    __import__("dotenv").load_dotenv()
    client = __import__("python_graphql_client").GraphqlClient(
        "https://api.github.com/graphql", dict(Authorization=F"token {os.environ['GITHUB_TOKEN']}")
    )

In [2]:
%%
    async def search_one(query, after=None):
        return await client.execute_async(
```graphql
{
      search(query: "%s", type: REPOSITORY, first: 100, after: %s) {
        repositoryCount
        wikiCount
        pageInfo {
            hasNextPage
            endCursor
        }
        edges {
          node {
            ... on Repository {
              url 
             stargazerCount
            id
              forkCount
              repositoryTopics(first: 20) {
                edges {
                  node {
                    topic {
                      name
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
```
        % (query, str(after and F'"{after}"' or "null")))
        

async def search_one(query, after=None):
 return await client.execute_async(
 
 { 
 search ( query : "%s" , type : REPOSITORY , first : 100 , after : % s ) { 
 repositoryCount 
 wikiCount 
 pageInfo { 
 hasNextPage 
 endCursor 
 } 
 edges { 
 node { 
 ... on Repository { 
 url 
 stargazerCount 
 id 
 forkCount 
 repositoryTopics ( first : 20 ) { 
 edges { 
 node { 
 topic { 
 name 
 } 
 } 
 } 
 } 
 } 
 } 
 } 
 } 
 } 
 
 
 % (query, str(after and F'"{after}"' or "null")))

In [3]:
%%
    async def search(query, stop=20):
[paginate through a search results](https://til.simonwillison.net/github/graphql-pagination-python) and collect the results.
        
        queries = [await search_one(query)]
        ct = 1
        while queries[-1]["data"]["search"]["pageInfo"]["hasNextPage"]:
            if ct > stop: break
            print(F"done {ct}")
            queries.append(await search_one(query, queries[-1]["data"]["search"]["pageInfo"]["endCursor"]))
            ct += 1
        return queries

async def search(query, stop=20):
 
 paginate through a search results and collect the results. 
 queries = [await search_one(query)]
 ct = 1
 while queries[-1]["data"]["search"]["pageInfo"]["hasNextPage"]:
 if ct > stop: break
 print(F"done {ct}")
 queries.append(await search_one(query, queries[-1]["data"]["search"]["pageInfo"]["endCursor"]))
 ct += 1
 return queries

In [4]:
    df = DataFrame(await search("assistive technology"))

done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9


In [5]:
g = df["data"].series().search.series().edges.explode().series().node.series().set_index("id")
g.join(
    g.repositoryTopics.series().edges.enumerate("entry").dropna().series().node.series().topic.series().name.groupby("id").agg(list).rename("tags")
).sort_values("stargazerCount").tail(20).style

,url,stargazerCount,forkCount,repositoryTopics,tags
id,,,,,
R_kgDOJhH1vA,https://github.com/Eloquent-Algorithmics/Automotive-AI,17,8,"{'edges': [{'node': {'topic': {'name': 'automotive'}}}, {'node': {'topic': {'name': 'automotive-software'}}}, {'node': {'topic': {'name': 'elm327'}}}, {'node': {'topic': {'name': 'gpt-3'}}}, {'node': {'topic': {'name': 'gpt-4'}}}, {'node': {'topic': {'name': 'obd-ii'}}}, {'node': {'topic': {'name': 'python'}}}, {'node': {'topic': {'name': 'chatgpt'}}}, {'node': {'topic': {'name': 'obd2'}}}, {'node': {'topic': {'name': 'obdii'}}}, {'node': {'topic': {'name': 'assistant'}}}, {'node': {'topic': {'name': 'chatgpt-api'}}}]}","['automotive', 'automotive-software', 'elm327', 'gpt-3', 'gpt-4', 'obd-ii', 'python', 'chatgpt', 'obd2', 'obdii', 'assistant', 'chatgpt-api']"
MDEwOlJlcG9zaXRvcnkyNDIxMzc2OTI=,https://github.com/abdallahkhairy/GP-Data_Analysis_and_ML,19,3,{'edges': []},nan
R_kgDOKncEmg,https://github.com/stuxctf/mhf,20,4,{'edges': []},nan
MDEwOlJlcG9zaXRvcnk0MzU4MDc3Ng==,https://github.com/WhatSock/w3c-alternative-text-computation,21,9,{'edges': []},nan
MDEwOlJlcG9zaXRvcnkxMTE0NzQ0NTk=,https://github.com/openassistive/awesome-assistivetech,24,2,"{'edges': [{'node': {'topic': {'name': 'awesome'}}}, {'node': {'topic': {'name': 'assistive-technology'}}}]}","['awesome', 'assistive-technology']"
MDEwOlJlcG9zaXRvcnk0Mjg0ODcy,https://github.com/tmcw/windchime,26,3,{'edges': []},nan
R_kgDOG1sIow,https://github.com/khachoang1412/HRRIS_for_MEC,29,8,{'edges': []},nan
R_kgDOGQZamg,https://github.com/w3c/at-driver,34,4,{'edges': []},nan
R_kgDOL2rM-Q,https://github.com/ruvnet/rUv-Enterprise-AI-Guide,37,6,{'edges': []},nan


In [6]:
    df = DataFrame(await search("screen reader"))

done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9


In [7]:
g = df["data"].series().search.series().edges.explode().series().node.series().set_index("id")

g.join(
    g.repositoryTopics.series().edges.enumerate("entry").dropna().series().node.series().topic.series().name.groupby("id").agg(list).rename("tags")
).sort_values("stargazerCount").tail(20).style

,url,stargazerCount,forkCount,repositoryTopics,tags
id,,,,,
R_kgDOGfs1qg,https://github.com/stardew-access/stardew-access,74,25,"{'edges': [{'node': {'topic': {'name': 'stardew-valley'}}}, {'node': {'topic': {'name': 'accessibility'}}}, {'node': {'topic': {'name': 'stardew-valley-mods'}}}]}","['stardew-valley', 'accessibility', 'stardew-valley-mods']"
MDEwOlJlcG9zaXRvcnkyNTUwOTIzMTY=,https://github.com/lightsoutgames/godot-accessibility,81,6,{'edges': []},nan
MDEwOlJlcG9zaXRvcnk3NDIyODYyMg==,https://github.com/tspivey/tdsr,83,20,{'edges': []},nan
R_kgDOJVXWcA,https://github.com/dhamaniasad/cytev2,89,43,{'edges': []},nan
MDEwOlJlcG9zaXRvcnkxNjIwNjQ4Mg==,https://github.com/dkager/tolk,95,42,{'edges': []},nan
R_kgDOLv_gtg,https://github.com/tongyx361/Awesome-LLM4Math,101,2,"{'edges': [{'node': {'topic': {'name': 'ai4math'}}}, {'node': {'topic': {'name': 'llm'}}}, {'node': {'topic': {'name': 'mathematics'}}}, {'node': {'topic': {'name': 'llm4math'}}}]}","['ai4math', 'llm', 'mathematics', 'llm4math']"
R_kgDOJmB7Gw,https://github.com/guidepup/virtual-screen-reader,106,3,"{'edges': [{'node': {'topic': {'name': 'a11y'}}}, {'node': {'topic': {'name': 'a11y-api'}}}, {'node': {'topic': {'name': 'a11y-automation'}}}, {'node': {'topic': {'name': 'a11y-testing'}}}, {'node': {'topic': {'name': 'accessibility'}}}, {'node': {'topic': {'name': 'accessibility-automation'}}}, {'node': {'topic': {'name': 'accessibility-testing'}}}, {'node': {'topic': {'name': 'screen-reader'}}}, {'node': {'topic': {'name': 'screen-reader-api'}}}]}","['a11y', 'a11y-api', 'a11y-automation', 'a11y-testing', 'accessibility', 'accessibility-automation', 'accessibility-testing', 'screen-reader', 'screen-reader-api']"
MDEwOlJlcG9zaXRvcnk4MTQ0MjI0OQ==,https://github.com/benbowes/react-responsive-select,109,17,"{'edges': [{'node': {'topic': {'name': 'react'}}}, {'node': {'topic': {'name': 'select'}}}, {'node': {'topic': {'name': 'dropdown'}}}, {'node': {'topic': {'name': 'responsive'}}}, {'node': {'topic': {'name': 'touch'}}}, {'node': {'topic': {'name': 'accessible'}}}]}","['react', 'select', 'dropdown', 'responsive', 'touch', 'accessible']"
R_kgDOLFQWEA,https://github.com/mzdk100/rigela,161,7,"{'edges': [{'node': {'topic': {'name': 'a11y'}}}, {'node': {'topic': {'name': 'accessibility'}}}, {'node': {'topic': {'name': 'blind'}}}, {'node': {'topic': {'name': 'jab'}}}, {'node': {'topic': {'name': 'java-access-bridge'}}}, {'node': {'topic': {'name': 'msaa'}}}, {'node': {'topic': {'name': 'nvda'}}}, {'node': {'topic': {'name': 'rsr'}}}, {'node': {'topic': {'name': 'rust'}}}, {'node': {'topic': {'name': 'screen-reader'}}}, {'node': {'topic': {'name': 'talkback'}}}, {'node': {'topic': {'name': 'ui-automation'}}}, {'node': {'topic': {'name': 'ia2'}}}, {'node': {'topic': {'name': 'iaccessible2'}}}]}","['a11y', 'accessibility', 'blind', 'jab', 'java-access-bridge', 'msaa', 'nvda', 'rsr', 'rust', 'screen-reader', 'talkback', 'ui-automation', 'ia2', 'iaccessible2']"


In [ ]:
    df = DataFrame(await search("assistive technology"))

done 1
done 2
done 3
done 4


In [ ]:
g = df["data"].series().search.series().edges.explode().series().node.series().set_index("id")

g.join(
    g.repositoryTopics.series().edges.enumerate("entry").dropna().series().node.series().topic.series().name.groupby("id").agg(list).rename("tags")
).sort_values("stargazerCount").head().style